In [1]:
pip install python-terrier

     |████████████████████████████████| 97 kB 2.0 MB/s 
     |████████████████████████████████| 1.1 MB 35.8 MB/s 
     |████████████████████████████████| 69 kB 3.6 MB/s 
     |████████████████████████████████| 255 kB 41.3 MB/s 
     |████████████████████████████████| 41 kB 244 kB/s 
     |████████████████████████████████| 6.4 MB 49.8 MB/s 
     |████████████████████████████████| 596 kB 41.3 MB/s 
     |████████████████████████████████| 1.2 MB 31.2 MB/s 
     |████████████████████████████████| 126 kB 37.5 MB/s 
     |████████████████████████████████| 72 kB 1.2 MB/s 
     |████████████████████████████████| 294 kB 55.7 MB/s 
     |████████████████████████████████| 291 kB 52.5 MB/s 
     |████████████████████████████████| 59 kB 4.8 MB/s 
     |████████████████████████████████| 45 kB 2.4 MB/s 
  Created wheel for python-terrier: filename=python_terrier-0.8.0-py3-none-any.whl size=103883 sha256=af6fbceefb810950a12b3c40922a8e8b3a83dbbbdb23eaf3ecc2bf4dfcdbfe5d
  Stored in directory: /root/.cac

In [2]:
import pyterrier as pt
if not pt.started():
    pt.init(mem=8000, version='snapshot', tqdm='notebook', 
            boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"]
           )

Done
terrier-python-helper 0.0.6 jar not found, downloading to /root/.pyterrier...
Done
terrier-prf -SNAPSHOT jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.8.0 has loaded Terrier 5.6 (built by jitpack on 2022-01-14 10:54)



In [3]:
dataset = pt.datasets.get_dataset("msmarco_passage")

In [4]:
indexref1 = dataset.get_index("terrier_stemmed")

data.direct.bf:   0%|          | 0.00/486M [00:00<?, ?iB/s]

data.document.fsarrayfile:   0%|          | 0.00/177M [00:00<?, ?iB/s]

data.inverted.bf:   0%|          | 0.00/377M [00:00<?, ?iB/s]

data.lexicon.fsomapfile:   0%|          | 0.00/100M [00:00<?, ?iB/s]

data.lexicon.fsomaphash:   0%|          | 0.00/0.99k [00:00<?, ?iB/s]

data.lexicon.fsomapid:   0%|          | 0.00/4.47M [00:00<?, ?iB/s]

data.meta.idx:   0%|          | 0.00/67.5M [00:00<?, ?iB/s]

data.meta.zdata:   0%|          | 0.00/193M [00:00<?, ?iB/s]

data.properties:   0%|          | 0.00/4.29k [00:00<?, ?iB/s]

md5sums:   0%|          | 0.00/480 [00:00<?, ?iB/s]

##Query Expansion Methods

In [20]:
pipelineKL = pt.BatchRetrieve(indexref1, wmodel="BM25", controls={"qemodel" : "Bo1", "qe" : "on"})
pipelineBo1 = pt.BatchRetrieve(indexref1, wmodel="BM25", controls={"qemodel" : "KL", "qe" : "on"})
pipelineRM3 = pt.BatchRetrieve(indexref1, wmodel="BM25", controls={"qemodel" : "RM3", "qe" : "on"})



##Collection Enrichment with Wikipedia index

In [21]:
bm25 = pt.BatchRetrieve(indexref1)
PipeWiki = bm25 >> pt.rewrite.stash_results(clear=False)>> pt.rewrite.RM3(indexref1)>> pt.rewrite.reset_results()>> bm25

In [22]:
topics = dataset.get_topics("test-2019")
qrels= dataset.get_qrels("test-2019")

##Comparing Baseline BM25 and QE & Enrichment scores

In [23]:
RM3 = pt.rewrite.RM3(indexref1)


In [17]:
pt.Experiment(
    [
      bm25, 
      pipelineBo1,
      pipelineKL,
      bm25 >> RM3 >> bm25, 
      #PipeWiki,
    ],
    topics,
    qrels,
    eval_metrics=['recip_rank','num_rel','num_ret','num_rel_ret','recall_10','recall_100','recall_1000','ndcg_cut_10','ndcg_cut_100'],
    names=["BM25", "+Bo1", "+KL","+RM3"] 
    )

,name,recip_rank,num_rel,num_ret,num_rel_ret,recall_10,recall_100,recall_1000,ndcg_cut_10,ndcg_cut_100
0,BM25,0.799962,4102.0,42005.0,2718.0,0.127234,0.425640,0.715164,0.502513,0.485995
1,+Bo1,0.788124,4102.0,43000.0,2980.0,0.128515,0.454161,0.751070,0.508620,0.500102
2,+KL,0.788124,4102.0,43000.0,2980.0,0.128515,0.454161,0.751070,0.508620,0.500102
3,+RM3,0.759464,4102.0,43000.0,3016.0,0.138888,0.462245,0.761899,0.523149,0.519726


In [24]:
pt.Experiment(
    [
      bm25, 
      pipelineBo1,
      pipelineKL,
      bm25 >> RM3 >> bm25, 
      PipeWiki,
    ],
    topics,
    qrels,
    eval_metrics=['recip_rank','num_rel','num_ret','num_rel_ret','recall_10','recall_100','recall_1000','ndcg_cut_10','ndcg_cut_100'],
    names=["BM25", "+Bo1", "+KL","+RM3","+Wiki_enrich"] 
    )

,name,recip_rank,num_rel,num_ret,num_rel_ret,recall_10,recall_100,recall_1000,ndcg_cut_10,ndcg_cut_100
0,BM25,0.799962,4102.0,42005.0,2718.0,0.127234,0.425640,0.715164,0.502513,0.485995
1,+Bo1,0.785023,4102.0,43000.0,2970.0,0.128436,0.452633,0.746565,0.505715,0.498427
2,+KL,0.788124,4102.0,43000.0,2980.0,0.128515,0.454161,0.751070,0.508620,0.500102
3,+RM3,0.759464,4102.0,43000.0,3016.0,0.138888,0.462245,0.761899,0.523149,0.519726
4,+Wiki_enrich,0.759464,4102.0,42005.0,2718.0,0.138888,0.462849,0.715164,0.523149,0.520067


In [31]:
rm3 = bm25 >> RM3 >> bm25
import pandas as pd
pd.options.display.max_colwidth = 700
rm3.search('lps laws definition')

,qid,docid,docno,rank,score,query_0,query
0,1,456708,456708,0,13.596846,lps laws definition,applypipeline:off 5^0.025154639 law^0.261030942 state^0.050309278 claus^0.037731957 section^0.025154639 standard^0.025154639 registr^0.025154639 definit^0.225154653 requir^0.025154639 lp^0.300000012
1,1,1465664,1465664,1,13.070251,lps laws definition,applypipeline:off 5^0.025154639 law^0.261030942 state^0.050309278 claus^0.037731957 section^0.025154639 standard^0.025154639 registr^0.025154639 definit^0.225154653 requir^0.025154639 lp^0.300000012
2,1,483935,483935,2,12.159928,lps laws definition,applypipeline:off 5^0.025154639 law^0.261030942 state^0.050309278 claus^0.037731957 section^0.025154639 standard^0.025154639 registr^0.025154639 definit^0.225154653 requir^0.025154639 lp^0.300000012
3,1,4774806,4774806,3,11.538109,lps laws definition,applypipeline:off 5^0.025154639 law^0.261030942 state^0.050309278 claus^0.037731957 section^0.025154639 standard^0.025154639 registr^0.025154639 definit^0.225154653 requir^0.025154639 lp^0.300000012
4,1,2490482,2490482,4,11.452818,lps laws definition,applypipeline:off 5^0.025154639 law^0.261030942 state^0.050309278 claus^0.037731957 section^0.025154639 standard^0.025154639 registr^0.025154639 definit^0.225154653 requir^0.025154639 lp^0.300000012
...,...,...,...,...,...,...,...
995,1,3886821,3886821,995,6.972904,lps laws definition,applypipeline:off 5^0.025154639 law^0.261030942 state^0.050309278 claus^0.037731957 section^0.025154639 standard^0.025154639 registr^0.025154639 definit^0.225154653 requir^0.025154639 lp^0.300000012
996,1,6813280,6813280,996,6.972904,lps laws definition,applypipeline:off 5^0.025154639 law^0.261030942 state^0.050309278 claus^0.037731957 section^0.025154639 standard^0.025154639 registr^0.025154639 definit^0.225154653 requir^0.025154639 lp^0.300000012
997,1,1972871,1972871,997,6.970115,lps laws definition,applypipeline:off 5^0.025154639 law^0.261030942 state^0.050309278 claus^0.037731957 section^0.025154639 standard^0.025154639 registr^0.025154639 definit^0.225154653 requir^0.025154639 lp^0.300000012
998,1,2900713,2900713,998,6.968544,lps laws definition,applypipeline:off 5^0.025154639 law^0.261030942 state^0.050309278 claus^0.037731957 section^0.025154639 standard^0.025154639 registr^0.025154639 definit^0.225154653 requir^0.025154639 lp^0.300000012


In [28]:
PipeWiki.search('when was salvation army founded')

,qid,docid,docno,rank,score,query
0,1,8689057,8689057,0,21.551640,applypipeline:off contact^0.063507490 armi^0.280703574 form^0.025403000 question^0.025403000 direct^0.025403000 salvat^0.280703574 assist^0.025403000 email^0.025403000 local^0.025403000 found^0.222667366
1,1,8689056,8689056,1,21.489036,applypipeline:off contact^0.063507490 armi^0.280703574 form^0.025403000 question^0.025403000 direct^0.025403000 salvat^0.280703574 assist^0.025403000 email^0.025403000 local^0.025403000 found^0.222667366
2,1,4511504,4511504,2,17.114608,applypipeline:off contact^0.063507490 armi^0.280703574 form^0.025403000 question^0.025403000 direct^0.025403000 salvat^0.280703574 assist^0.025403000 email^0.025403000 local^0.025403000 found^0.222667366
3,1,5427063,5427063,3,17.098491,applypipeline:off contact^0.063507490 armi^0.280703574 form^0.025403000 question^0.025403000 direct^0.025403000 salvat^0.280703574 assist^0.025403000 email^0.025403000 local^0.025403000 found^0.222667366
4,1,4511501,4511501,4,17.034418,applypipeline:off contact^0.063507490 armi^0.280703574 form^0.025403000 question^0.025403000 direct^0.025403000 salvat^0.280703574 assist^0.025403000 email^0.025403000 local^0.025403000 found^0.222667366
...,...,...,...,...,...,...
995,1,3220631,3220631,995,6.680187,applypipeline:off contact^0.063507490 armi^0.280703574 form^0.025403000 question^0.025403000 direct^0.025403000 salvat^0.280703574 assist^0.025403000 email^0.025403000 local^0.025403000 found^0.222667366
996,1,4313722,4313722,996,6.680187,applypipeline:off contact^0.063507490 armi^0.280703574 form^0.025403000 question^0.025403000 direct^0.025403000 salvat^0.280703574 assist^0.025403000 email^0.025403000 local^0.025403000 found^0.222667366
997,1,4561654,4561654,997,6.680187,applypipeline:off contact^0.063507490 armi^0.280703574 form^0.025403000 question^0.025403000 direct^0.025403000 salvat^0.280703574 assist^0.025403000 email^0.025403000 local^0.025403000 found^0.222667366
998,1,6799563,6799563,998,6.679195,applypipeline:off contact^0.063507490 armi^0.280703574 form^0.025403000 question^0.025403000 direct^0.025403000 salvat^0.280703574 assist^0.025403000 email^0.025403000 local^0.025403000 found^0.222667366


##How many queries improved

In [32]:
bm25_exp = pt.Experiment(
    [bm25],
    topics,
    qrels,
    filter_by_qrels=True,
    perquery=True,
    eval_metrics=['ndcg_cut_10'],
)

In [33]:
bm25rm3_exp = pt.Experiment(
    [rm3],
    topics,
    qrels,
    filter_by_qrels=True,
    perquery=True,
    eval_metrics=['ndcg_cut_10'],
)

In [35]:
bm25_exp

,name,qid,measure,value
9,BR(DPH),1037798,ndcg_cut_10,0.305733
21,BR(DPH),104861,ndcg_cut_10,0.666706
2,BR(DPH),1063750,ndcg_cut_10,0.000000
29,BR(DPH),1103812,ndcg_cut_10,0.591745
26,BR(DPH),1106007,ndcg_cut_10,0.000000
1,BR(DPH),1110199,ndcg_cut_10,0.368829
19,BR(DPH),1112341,ndcg_cut_10,0.620551
40,BR(DPH),1113437,ndcg_cut_10,0.140898
35,BR(DPH),1114646,ndcg_cut_10,0.414038
24,BR(DPH),1114819,ndcg_cut_10,0.840563


In [43]:
merged = pd.merge(bm25_exp, bm25rm3_exp,on = 'qid')
compare = pd.merge(merged, topics,on = 'qid')
compare.sort_values(by = ['value_x'])

,name_x,qid,measure_x,value_x,name_y,measure_y,value_y,query
2,BR(DPH),1063750,ndcg_cut_10,0.000000,"Compose(Compose(BR(DPH), <pyterrier.rewrite.RM3 object at 0x7f4c91c2ad50>), BR(DPH))",ndcg_cut_10,0.000000,why did the us volunterilay enter ww1
4,BR(DPH),1106007,ndcg_cut_10,0.000000,"Compose(Compose(BR(DPH), <pyterrier.rewrite.RM3 object at 0x7f4c91c2ad50>), BR(DPH))",ndcg_cut_10,0.000000,define visceral
13,BR(DPH),1121709,ndcg_cut_10,0.000000,"Compose(Compose(BR(DPH), <pyterrier.rewrite.RM3 object at 0x7f4c91c2ad50>), BR(DPH))",ndcg_cut_10,0.000000,what are the three percenters
31,BR(DPH),451602,ndcg_cut_10,0.112102,"Compose(Compose(BR(DPH), <pyterrier.rewrite.RM3 object at 0x7f4c91c2ad50>), BR(DPH))",ndcg_cut_10,0.116404,medicare s definition of mechanical ventilation
7,BR(DPH),1113437,ndcg_cut_10,0.140898,"Compose(Compose(BR(DPH), <pyterrier.rewrite.RM3 object at 0x7f4c91c2ad50>), BR(DPH))",ndcg_cut_10,0.195512,what is physical description of spruce
30,BR(DPH),443396,ndcg_cut_10,0.142019,"Compose(Compose(BR(DPH), <pyterrier.rewrite.RM3 object at 0x7f4c91c2ad50>), BR(DPH))",ndcg_cut_10,0.142019,lps laws definition
29,BR(DPH),405717,ndcg_cut_10,0.145190,"Compose(Compose(BR(DPH), <pyterrier.rewrite.RM3 object at 0x7f4c91c2ad50>), BR(DPH))",ndcg_cut_10,0.207292,is cdg airport in main paris
42,BR(DPH),962179,ndcg_cut_10,0.151762,"Compose(Compose(BR(DPH), <pyterrier.rewrite.RM3 object at 0x7f4c91c2ad50>), BR(DPH))",ndcg_cut_10,0.063621,when was the salvation army founded
41,BR(DPH),915593,ndcg_cut_10,0.249363,"Compose(Compose(BR(DPH), <pyterrier.rewrite.RM3 object at 0x7f4c91c2ad50>), BR(DPH))",ndcg_cut_10,0.042414,what types of food can you cook sous vide
0,BR(DPH),1037798,ndcg_cut_10,0.305733,"Compose(Compose(BR(DPH), <pyterrier.rewrite.RM3 object at 0x7f4c91c2ad50>), BR(DPH))",ndcg_cut_10,0.438473,who is robert gray


Worst 5 performing queries on BM25 baseline did not improve but there is an overall improvement in ndcg@10. 

In total for 25 out 43 queries ndcg_10 scores improved.

In [42]:

#improved queries out of 43

compare[compare.value_y > compare.value_x ].shape[0]

25